In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Read metadata file
data_paths = {'appliences':'/kaggle/input/appliences/appliences/','urban':'/kaggle/input/urbansound8k/'}
meta1 = data_paths['appliences'] + 'metadata/appliences.csv'
meta2 = data_paths['urban'] + 'UrbanSound8K.csv'
df_UrbanSound = pd.read_csv(meta2)
df_Appliences = pd.read_csv(meta1)
upsmpl = int(df_UrbanSound.classID.value_counts().max() / df_Appliences.classID.value_counts().min())
df_Appliences = df_Appliences.sample(frac=upsmpl, replace=True, random_state=1)
df_UrbanSound['relative_path'] = '/fold' + df_UrbanSound['fold'].astype(str) + '/' + df_UrbanSound['slice_file_name'].astype(str)
df_Appliences['relative_path'] = '/' + df_Appliences['fold'].astype(str) + '/' + df_Appliences['slice_file_name'].astype(str)

df = pd.concat([df_UrbanSound,df_Appliences])
df = df.reset_index(drop=True)
df.head()

#Take relative columns
df = df[['relative_path', 'classID']]
df.head()
#print((df.classID==1).sum())


,relative_path,classID
0,/fold5/100032-3-0-0.wav,3
1,/fold5/100263-2-0-117.wav,2
2,/fold5/100263-2-0-121.wav,2
3,/fold5/100263-2-0-126.wav,2
4,/fold5/100263-2-0-137.wav,2


In [8]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import matplotlib
import matplotlib.pyplot as plt

class AudioUtil():
    #---------------
    # Load an audio file. Return the signal as a tensor and the sample rate
    #---------------
    @staticmethod
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)
    
    #---------------
    # Convert soundfile to desired number of channels
    #---------------
    @staticmethod
    def rechannel(aud, new_channel):
        
        sig, sr = aud
        
        if sig.shape[0] == new_channel:
            #Nothing todo
            return aud
        
        if (new_channel == 1):
            #Convert stereo to mono by selecting only the first channel
            resig = sig[:1, :]
        else:
            #Convert from mono to sterio by duplicating the first channel
            resig = torch.cat([sig,sig])
        return ((resig, sr))
    
    #---------------
    #Resample to make sure samplerate is the same for all files - resample applies to one channel at a time
    #---------------
    @staticmethod
    def resample(aud, newsr):
        
        sig, sr = aud
        
        if (sr == newsr):
            #do nothing
            return aud
        
        num_channels = sig.shape[0]
        
        #resample first channel
        resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
        
        if (num_channels > 1):
            #Resample the second channel and merge both
            retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
            resig = torch.cat([resig, retwo])
            
        return ((resig, newsr))
    
    
    #-----------------
    #Pad or turncate the signal to be off a standard length in milliseconds
    #-----------------
    @staticmethod
    def pad_trunc(aud, max_ms):

        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr//1000 * max_ms
        
        if (sig_len > max_len):
            #Turncate the signal to the given length
            sig = sig[:,:max_len]
        elif (sig_len < max_len):
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len
            
            #pad with zeroes
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))
            
            sig = torch.cat((pad_begin, sig, pad_end), 1)
        
        return (sig, sr)

    #--------------------
    #Shift the signal by a random bit, end of signal is wrapped around 
    #to beginning
    #--------------------
    @staticmethod
    def time_shift(aud, shift_limit):
        sig, sr = aud
        
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)
    
    #----------------------------------
    #Genetate spectrogram
    #----------------------------------
    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig,sr = aud
        top_db = 80
        
        #spec has shape [channel, n_mels, time]
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
        
        #convert to db
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        
        return spec
    
    
    #Augment the spectrogram by masking out some sections of it in both the frequency
    #dimencion (Horizontal) and the time dimension (vertical bars)
    
    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec
        
        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)
        
        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)
        
        return np.absolute(aug_spec) # Jonas added np.absolute
    
        

In [41]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio

#----------------
#Sound dataset
#----------------
    
class SoundDS(Dataset):
    def __init__(self, df, data_paths):
        self.df = df
        self.data_paths = data_paths
        self.duration = 4000
        self.sr = 44100
        self.channel = 2
        self.shift_pct = 0.4
    
    #------------------
    #Number of items in dataset
    #------------------
    def __len__(self):
        return len(self.df)

    #------------------
    #Get i'th item in dataset
    #------------------
    def __getitem__(self, idx):

        # Get the class ID  
        class_id = self.df.loc[idx, 'classID']
        # Absolute file path of the audio file - concatenate the audio direcory
        # with the relative path
        if class_id <= df_UrbanSound['classID'].max():  # UrbanSound data set (classIDs 0 - 10)
            audio_file = self.data_paths['urban'] + self.df.loc[idx, 'relative_path']
        else: # Appliences (Class IDs => 10)
            audio_file = self.data_paths['appliences'] + self.df.loc[idx, 'relative_path']
              
        aud = AudioUtil.open(audio_file)
        #Make all the sounds have the same number of channels and same sample rate
        #Then make all samples the same length
        reaud = AudioUtil.resample(aud, self.sr)
        rechan = AudioUtil.rechannel(reaud, self.channel)
        
        dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
        shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
        sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
        #aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
        aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)
        return aug_sgram, class_id


In [42]:
# Splitt data for training and validation

myds = SoundDS(df, data_paths) #returns spectrogram and classID
#Random split between training and validation data

num_items = len(myds)
num_train = round(num_items*0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)


In [15]:
import torch.nn.functional as F
from torch.nn import init
from torch import nn

#---------------------------
# Audio Classification Model
#---------------------------
class AudioClassifier (nn.Module):
    #---------------------------
    #Build the model architecture
    #---------------------------
    
    def __init__(self):
        super().__init__()
        conv_layers = []
        
        #First convolution block with Relu and Batch Norm. Use Kaiming Initialisation
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        #Second convolution block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]
        
        #Third convolution block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        #Fourth convolution block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4] 
        
        # Linear classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=df.classID.max()+1) #Changed Jonas out_features
        
        # Wrap the convolutional blocks
        self.conv = nn.Sequential(*conv_layers)
    
    #-------------------------
    #Forward pass computations
    #-------------------------
    def forward(self, x):
        # Run convolutional blocks
        x = self.conv(x)
        
        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        
        # Linear layer
        x = self.lin(x)
        
        # Final output
        
        return x

# Create the model and put it on the GPU if available
myModel =  AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
#Check that it is on Cuda
next(myModel.parameters()).device

device(type='cpu')

In [16]:
#-------------------
#Training loop
#-------------------
def training(model, train_dl, num_epochs):
    #Loss function optimiser and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimiser, max_lr=0.001,
                                                    steps_per_epoch=int(len(train_dl)),
                                                    epochs=num_epochs,
                                                    anneal_strategy='linear')
    
    #Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0
        
        #repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
            
            #Normalise the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s
            
            #Zero prameter gradients
            optimiser.zero_grad()
            
            #Forward + backward + optimise
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimiser.step()
            scheduler.step()
            
            #Keep stats for Loss and Accuracy
            running_loss += loss.item()
            
            
            #Get predicted class with highest score
            _, prediction = torch.max(outputs, 1)
            # Count the predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]
            
            if i % 10 == 0: #
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
        
        #Print stats at the end of each epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction / total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')
    
    print('Finished Training')
    
num_epochs = 2
training(myModel, train_dl, num_epochs)
    

[1,     1] loss: 0.254
[1,    11] loss: 2.765
[1,    21] loss: 5.275
[1,    31] loss: 7.735
[1,    41] loss: 10.204
[1,    51] loss: 12.621
[1,    61] loss: 15.024
[1,    71] loss: 17.380
[1,    81] loss: 19.611
[1,    91] loss: 21.876
[1,   101] loss: 24.113
[1,   111] loss: 26.243
[1,   121] loss: 28.354
[1,   131] loss: 30.315
[1,   141] loss: 32.455
[1,   151] loss: 34.425
[1,   161] loss: 36.378
[1,   171] loss: 38.364
[1,   181] loss: 40.125
[1,   191] loss: 42.013
[1,   201] loss: 43.817
[1,   211] loss: 45.600
[1,   221] loss: 47.244
[1,   231] loss: 48.986
[1,   241] loss: 50.659
[1,   251] loss: 52.335
[1,   261] loss: 54.031
[1,   271] loss: 55.760
[1,   281] loss: 57.478
[1,   291] loss: 59.157
[1,   301] loss: 60.754
[1,   311] loss: 62.368
[1,   321] loss: 63.912
[1,   331] loss: 65.508
[1,   341] loss: 67.097
[1,   351] loss: 68.571
[1,   361] loss: 70.130
[1,   371] loss: 71.619
[1,   381] loss: 73.103
[1,   391] loss: 74.660
[1,   401] loss: 76.077
[1,   411] loss: 77.

In [43]:
import numpy as np

#-------------------
#Inferrence
#-------------------

def inferrence (model, v_dl):
    correct_prediction = 0
    total_prediction = 0
    
    #Disable gradient updates
    with torch.no_grad():
        for data in v_dl:
            # Get the input features and target labels and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
            #Normalise the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s
            
            #Get predictions
            outputs = model(inputs)
            
            #Get predicted putputs with highest score
            i, prediction = torch.max(outputs,1)
            for n in range(len(outputs)):
                if labels[n].item() == -1:
                    print("Predicted class:{}, actual class:{}".format(prediction[n].item(), labels[n].item()))
                else:
                    if prediction[n].item() == labels[n].item():
                        df_metr['correct_pred'].loc[prediction[n].item()] += 1
                    else:
                        df_metr['false_pred'].loc[prediction[n].item()] += 1
                #print("Predicted class:{}, score:{}, actual class:{}".format(prediction[n].item(), outputs[n].max().item(), labels[n].item()))
            #print(outputs[15].max(),prediction[15], labels[15])
            #Count of predictions that matched target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]
            
    acc = correct_prediction / total_prediction
    df_metr['accuracy'] = 1 - (df_metr['false_pred'] / df_metr['correct_pred'])
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')



    
    
classIDs = sorted(list(df.classID.unique()))
df_metr = pd.DataFrame({'classIDs': classIDs,
                        'correct_pred': np.zeros(len(classIDs)),
                        'false_pred': np.zeros(len(classIDs)),
                        'accuracy': np.zeros(len(classIDs))})
df_metr = df_metr.set_index('classIDs')

#Run inferrence on trained model with validation set
inferrence(myModel, val_dl)


          correct_pred  false_pred  accuracy
classIDs                                    
0                  0.0         0.0       0.0
1                  0.0         0.0       0.0
2                  0.0         0.0       0.0
3                  0.0         0.0       0.0
4                  0.0         0.0       0.0
5                  0.0         0.0       0.0
6                  0.0         0.0       0.0
7                  0.0         0.0       0.0
8                  0.0         0.0       0.0
9                  0.0         0.0       0.0
10                 0.0         0.0       0.0
11                 0.0         0.0       0.0
12                 0.0         0.0       0.0
Accuracy: 0.60, Total items: 2592


In [44]:
print(df_metr)

          correct_pred  false_pred  accuracy
classIDs                                    
0                 68.0        80.0 -0.176471
1                 33.0        41.0 -0.242424
2                 76.0       140.0 -0.842105
3                 92.0       135.0 -0.467391
4                 60.0        87.0 -0.450000
5                111.0       140.0 -0.261261
6                 19.0        20.0 -0.052632
7                126.0       136.0 -0.079365
8                103.0       101.0  0.019417
9                 65.0       112.0 -0.723077
10               273.0        11.0  0.959707
11               348.0        11.0  0.968391
12               186.0        18.0  0.903226


In [25]:
df.loc[5284,:]

relative_path    /fold5/203654-9-0-42.wav
classID                                 9
Name: 5284, dtype: object